In [1]:
import numpy as np
import pandas as pd
##### WEB scriping #####
import requests
from bs4 import BeautifulSoup
########################
from datetime import datetime, date, timedelta
from googletrans import Translator 

<font style="font-size: 80px"><font color="yellow">C</font>onsume <font color="yellow">P</font>roduct <font color="yellow">I</font>ndex (<font color="yellow">CPI</font>)</font>

In [2]:
'''
url = 'https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=csv'
local_path = 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454.csv'

response = requests.get(url)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f'Download success, status code {response.status_code}')
else:
    print(f'Download error, status code {response.status_code}')

path = 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454.csv'
'''

"\nurl = 'https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=csv'\nlocal_path = 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454.csv'\n\nresponse = requests.get(url)\n\nif(response.status_code == 200):\n    with open(local_path, 'wb') as file:\n        file.write(response.content)\n    print(f'Download success, status code {response.status_code}')\nelse:\n    print(f'Download error, status code {response.status_code}')\n\npath = 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_228454.csv'\n"

In [3]:
CPIworld = pd.read_csv("../../CSV crudo/CPIworld.csv")
CPIworld

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN
1,NaN,AFE,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.750981,5.370290,5.250171,6.571396,6.399343,4.720811,4.120246,6.362961,6.079328,NaN
2,Afganistán,AFG,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
3,NaN,AFW,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.439201,1.758052,2.130268,1.494564,1.764635,1.784050,1.758565,2.437609,3.837868,NaN
4,Angola,AGO,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.767324,0.428958,-0.536929,0.273169,1.488234,1.053798,2.675992,0.198228,3.353691,NaN
262,"Yemen, Rep. del",YEM,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,Sudáfrica,ZAF,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,1.288859,2.102374,1.246285,1.33797,2.534973,4.069029,...,5.784469,6.129838,4.540642,6.571396,5.184247,4.517165,4.120246,3.210036,4.611672,NaN
264,Zambia,ZMB,"Inflación, precios al consumidor (% anual)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,6.977676,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.732585,22.021234,NaN


### Columns title

In [4]:
CPIworld.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 66"], inplace=True)
CPIworld.rename(columns={"Country Name":"Country", "Country Code":"Code"}, inplace=True)
CPIworld

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.627472,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN
1,NaN,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.158707,5.750981,5.370290,5.250171,6.571396,6.399343,4.720811,4.120246,6.362961,6.079328
2,Afganistán,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.441213,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN
3,NaN,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.578375,2.439201,1.758052,2.130268,1.494564,1.764635,1.784050,1.758565,2.437609,3.837868
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.277905,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.476738,1.767324,0.428958,-0.536929,0.273169,1.488234,1.053798,2.675992,0.198228,3.353691
262,"Yemen, Rep. del",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.885387,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,Sudáfrica,ZAF,1.288859,2.102374,1.246285,1.33797,2.534973,4.069029,3.489234,3.538992,...,5.724658,5.784469,6.129838,4.540642,6.571396,5.184247,4.517165,4.120246,3.210036,4.611672
264,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.575900,6.977676,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.732585,22.021234


In [5]:
country_check = {}
for i in CPIworld["Country"]:
    if country_check.get(i):
        country_check[i] += 1
    else:
        country_check[i] = 1
        
country_repeat = {k: v for k, v in country_check.items() if(v>=2)}
country_repeat

{nan: 2}

In [6]:
CPIworld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 64 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  264 non-null    object 
 1   Code     266 non-null    object 
 2   1960     70 non-null     float64
 3   1961     72 non-null     float64
 4   1962     74 non-null     float64
 5   1963     75 non-null     float64
 6   1964     79 non-null     float64
 7   1965     86 non-null     float64
 8   1966     93 non-null     float64
 9   1967     100 non-null    float64
 10  1968     101 non-null    float64
 11  1969     102 non-null    float64
 12  1970     107 non-null    float64
 13  1971     111 non-null    float64
 14  1972     114 non-null    float64
 15  1973     117 non-null    float64
 16  1974     120 non-null    float64
 17  1975     124 non-null    float64
 18  1976     125 non-null    float64
 19  1977     129 non-null    float64
 20  1978     129 non-null    float64
 21  1979     124 non

## <font style="font-style:italic "><font color="#6CACE4">This data-set</font> <font color="white">not have Argentina</font> <font color="yellow">data.That</font> <font color="white">is my country</font> <font color="#6CACE4">where I born.</font></font> <font style="font-size:15px">(WEB scriping)</font>

In [7]:
print(CPIworld[CPIworld["Country"] == "Argentina"].iloc[:, 2:].isna().any().value_counts(normalize=True)) #False = 0 => All is nan
CPIworld.drop(index=CPIworld[CPIworld["Country"] == "Argentina" ].index.values, inplace=True)

True    1.0
Name: proportion, dtype: float64


### WEB scriping

In [8]:
url = "https://www.datosmundial.com/america/argentina/inflacion.php"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

tr_items = soup.find_all('td')

In [9]:
Argentina_inflation = {}
count_year = 0
count_Argentina = 1

for td in tr_items:
    if(count_year == 0):
        actual_year = td.text
        count_year = 5

    elif(count_Argentina == 0):
        Argentina_inflation[actual_year] = td.text
        count_Argentina = 5

    count_year -= 1
    count_Argentina -= 1

Argentina_inflation

{'2022': '94,80 %',
 '2021': '48,41 %',
 '2020': '42,02 %',
 '2019': '53,55 %',
 '2018': '34,28 %',
 '2017': '25,68 %',
 '2016': '26,50 %',
 '2014': '23,90 %',
 '2013': '10,62 %',
 '2012': '10,03 %',
 '2011': '9,47 %',
 '2010': '10,78 %',
 '2009': '6,28 %',
 '2008': '8,58 %',
 '2007': '8,83 %',
 '2006': '10,90 %',
 '2005': '9,64 %',
 '2004': '4,42 %',
 '2003': '13,44 %',
 '2002': '25,87 %',
 '2001': '-1,07 %',
 '2000': '-0,94 %',
 '1999': '-1,17 %',
 '1998': '0,92 %',
 '1997': '0,53 %',
 '1996': '0,16 %',
 '1995': '3,38 %',
 '1994': '4,18 %',
 '1993': '10,61 %',
 '1992': '24,90 %',
 '1991': '171,67 %',
 '1990': '2.313,96 %',
 '1989': '3.079,81 %',
 '1988': '342,96 %',
 '1987': '131,33 %',
 '1986': '90,10 %',
 '1985': '672,18 %',
 '1984': '626,72 %',
 '1983': '343,81 %',
 '1982': '164,78 %',
 '1981': '104,48 %',
 '1980': '100,76 %'}

In [10]:
dict_aux = {}
for key, value in Argentina_inflation.items():
    dict_aux[key] = float(value.split("%")[0].replace(".", "").replace(",", "."))   #How to change "2.313,98"

del dict_aux["2022"]
dict_aux

{'2021': 48.41,
 '2020': 42.02,
 '2019': 53.55,
 '2018': 34.28,
 '2017': 25.68,
 '2016': 26.5,
 '2014': 23.9,
 '2013': 10.62,
 '2012': 10.03,
 '2011': 9.47,
 '2010': 10.78,
 '2009': 6.28,
 '2008': 8.58,
 '2007': 8.83,
 '2006': 10.9,
 '2005': 9.64,
 '2004': 4.42,
 '2003': 13.44,
 '2002': 25.87,
 '2001': -1.07,
 '2000': -0.94,
 '1999': -1.17,
 '1998': 0.92,
 '1997': 0.53,
 '1996': 0.16,
 '1995': 3.38,
 '1994': 4.18,
 '1993': 10.61,
 '1992': 24.9,
 '1991': 171.67,
 '1990': 2313.96,
 '1989': 3079.81,
 '1988': 342.96,
 '1987': 131.33,
 '1986': 90.1,
 '1985': 672.18,
 '1984': 626.72,
 '1983': 343.81,
 '1982': 164.78,
 '1981': 104.48,
 '1980': 100.76}

In [11]:
addRow = pd.DataFrame([dict_aux])
addRow = addRow.assign(Country="Argentina", Code="ARG")
print(CPIworld.info())
print(addRow.info())

df_test =pd.concat([CPIworld, addRow], ignore_index=True)
#******
false_data = df_test[df_test["Country"] == "Argentina"].iloc[:, 22:].isna().any()
print("No data years: ", false_data.index[false_data].values)
#******
CPIworld = df_test

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, 0 to 265
Data columns (total 64 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  263 non-null    object 
 1   Code     265 non-null    object 
 2   1960     70 non-null     float64
 3   1961     72 non-null     float64
 4   1962     74 non-null     float64
 5   1963     75 non-null     float64
 6   1964     79 non-null     float64
 7   1965     86 non-null     float64
 8   1966     93 non-null     float64
 9   1967     100 non-null    float64
 10  1968     101 non-null    float64
 11  1969     102 non-null    float64
 12  1970     107 non-null    float64
 13  1971     111 non-null    float64
 14  1972     114 non-null    float64
 15  1973     117 non-null    float64
 16  1974     120 non-null    float64
 17  1975     124 non-null    float64
 18  1976     125 non-null    float64
 19  1977     129 non-null    float64
 20  1978     129 non-null    float64
 21  1979     124 non-null

In [12]:
CPIworld

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.627472,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN
1,NaN,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.158707,5.750981,5.370290,5.250171,6.571396,6.399343,4.720811,4.120246,6.362961,6.079328
2,Afganistán,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.441213,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN
3,NaN,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.578375,2.439201,1.758052,2.130268,1.494564,1.764635,1.784050,1.758565,2.437609,3.837868
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.277905,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,"Yemen, Rep. del",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.885387,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,Sudáfrica,ZAF,1.288859,2.102374,1.246285,1.33797,2.534973,4.069029,3.489234,3.538992,...,5.724658,5.784469,6.129838,4.540642,6.571396,5.184247,4.517165,4.120246,3.210036,4.611672
263,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.575900,6.977676,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.732585,22.021234
264,Zimbabwe,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.725327,1.634950,-0.197785,-2.430968,-1.543670,0.893962,10.618866,255.304991,557.201817,98.546105


In [13]:
int(CPIworld[CPIworld["Country"] == "Argentina"].index.values)

C:\Users\matia\AppData\Local\Temp\ipykernel_27640\3926330579.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int(CPIworld[CPIworld["Country"] == "Argentina"].index.values)


265

### NaN content

In [14]:
dataDelete = CPIworld[CPIworld["Country"].isna()==True]
print(dataDelete.iloc[:, :2], "\n", dataDelete.index)
CPIworld.drop(index=dataDelete.index, inplace=True) #index=[1, 3]
CPIworld.reset_index(inplace=True, drop=True)

  Country Code
1     NaN  AFE
3     NaN  AFW 
 Index([1, 3], dtype='int64')


In [15]:
noData = []
for i in CPIworld["Country"]:
    rowSelected = CPIworld[CPIworld["Country"] == i]
    
    if(rowSelected.iloc[:, 2:].isna().any().any()):
        TrueCount = rowSelected.iloc[:, 2:].isna().any().value_counts(normalize=True)[True]
        print(rowSelected.index.values, i, "have a", (TrueCount*100).round(2), "%", "NaN values")
        if(TrueCount == 1):
            noData.append(int(rowSelected.index.values))
        else:
            pass
    else:
        pass
print(len(noData), noData)

CPIworld.drop(index=noData, inplace=True)
CPIworld.reset_index(inplace=True, drop=True)
CPIworld

[0] Aruba have a 43.55 % NaN values
[1] Afganistán have a 75.81 % NaN values
[2] Angola have a 50.0 % NaN values
[3] Albania have a 51.61 % NaN values
[4] Andorra have a 100.0 % NaN values
[5] El mundo árabe have a 22.58 % NaN values
[6] Emiratos Árabes Unidos have a 79.03 % NaN values
[7] Armenia have a 54.84 % NaN values
[8] Samoa Americana have a 100.0 % NaN values
[9] Antigua y Barbuda have a 62.9 % NaN values
[12] Azerbaiyán have a 51.61 % NaN values
[13] Burundi have a 9.68 % NaN values
[15] Benin have a 53.23 % NaN values
[17] Bangladesh have a 43.55 % NaN values
[18] Bulgaria have a 41.94 % NaN values
[19] Bahrein have a 9.68 % NaN values
[20] Bahamas have a 11.29 % NaN values
[21] Bosnia y Herzegovina have a 74.19 % NaN values
[22] Belarús have a 53.23 % NaN values
[23] Belice have a 33.87 % NaN values
[24] Bermudas have a 100.0 % NaN values
[26] Brasil have a 33.87 % NaN values
[27] Barbados have a 14.52 % NaN values
[28] Brunei Darussalam have a 33.87 % NaN values
[29] Bhutá

C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.values))
C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.values))
C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.valu

[131] América Latina y el Caribe have a 11.29 % NaN values
[132] Países menos desarrollados: clasificación de las Naciones Unidas have a 41.94 % NaN values
[133] Países de ingreso bajo have a 38.71 % NaN values
[134] Liechtenstein have a 100.0 % NaN values
[136] Países de ingreso mediano bajo have a 22.58 % NaN values
[137] Ingreso mediano y bajo have a 33.87 % NaN values
[138] Lesotho have a 27.42 % NaN values
[139] avanzada del dividendo demográfico have a 43.55 % NaN values
[140] Lituania have a 51.61 % NaN values
[142] Letonia have a 51.61 % NaN values
[143] Región Administrativa Especial de Macao, China have a 48.39 % NaN values
[144] Isla de San Martín (parte francesa) have a 100.0 % NaN values
[146] Mónaco have a 100.0 % NaN values
[147] República de Moldova have a 51.61 % NaN values
[148] Madagascar have a 8.06 % NaN values
[149] Maldivas have a 41.94 % NaN values
[150] Oriente Medio y Norte de África have a 16.13 % NaN values
[152] Islas Marshall have a 100.0 % NaN values
[153

C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.values))
C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.values))
C:\Users\matia\AppData\Local\Temp\ipykernel_27640\4174831497.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  noData.append(int(rowSelected.index.valu

[211] Serbia have a 56.45 % NaN values
[212] África al sur del Sahara (excluido altos ingresos) have a 33.87 % NaN values
[213] Sudán del Sur have a 79.03 % NaN values
[214] África al sur del Sahara have a 24.19 % NaN values
[215] Pequeños Estados have a 29.03 % NaN values
[216] Santo Tomé y Príncipe have a 64.52 % NaN values
[217] Suriname have a 3.23 % NaN values
[218] República Eslovaca have a 51.61 % NaN values
[219] Eslovenia have a 33.87 % NaN values
[221] Eswatini have a 12.9 % NaN values
[222] Sint Maarten (Dutch part) have a 80.65 % NaN values
[223] Seychelles have a 19.35 % NaN values
[224] República Árabe Siria have a 14.52 % NaN values
[225] Islas Turcas y Caicos have a 100.0 % NaN values
[226] Chad have a 38.71 % NaN values
[227] Asia oriental y el Pacífico (BIRF y la AIF) have a 43.55 % NaN values
[228] Europa y Asia central (BIRF y la AIF) have a 54.84 % NaN values
[229] Togo have a 12.9 % NaN values
[231] Tayikistán have a 74.19 % NaN values
[232] Turkmenistán have a 10

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.627472,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN
1,Afganistán,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.441213,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN
2,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.277905,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266
3,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.031593,1.937621,1.625865,3.501208,-0.367317,2.060598,2.028060,1.411091,1.620887,2.041472
4,El mundo árabe,ARB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.611844,3.238108,2.773516,1.814077,2.068840,1.966826,2.458142,1.091848,1.777408,3.423629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,"Yemen, Rep. del",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.885387,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,Sudáfrica,ZAF,1.288859,2.102374,1.246285,1.33797,2.534973,4.069029,3.489234,3.538992,...,5.724658,5.784469,6.129838,4.540642,6.571396,5.184247,4.517165,4.120246,3.210036,4.611672
236,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.575900,6.977676,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.732585,22.021234
237,Zimbabwe,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.725327,1.634950,-0.197785,-2.430968,-1.543670,0.893962,10.618866,255.304991,557.201817,98.546105


In [16]:
nanCountry = {}

for i in CPIworld["Country"]:
    nanCountry[i] = CPIworld[CPIworld["Country"] == i].iloc[:, 2:].isna().any().value_counts(normalize=True)[False]
    nanCountry[i] *= 100

nanCountryOrder = dict(sorted(nanCountry.items(), key=lambda items: items[1], reverse=True))
nanCountryOrder

{'Australia': 100.0,
 'Austria': 100.0,
 'Bélgica': 100.0,
 'Burkina Faso': 100.0,
 'Bolivia': 100.0,
 'Canadá': 100.0,
 'Suiza': 100.0,
 'Colombia': 100.0,
 'Costa Rica': 100.0,
 'Chipre': 100.0,
 'Alemania': 100.0,
 'Dinamarca': 100.0,
 'República Dominicana': 100.0,
 'Ecuador': 100.0,
 'Egipto, República Árabe de': 100.0,
 'Zona del Euro': 100.0,
 'España': 100.0,
 'Unión Europea': 100.0,
 'Finlandia': 100.0,
 'Francia': 100.0,
 'Reino Unido': 100.0,
 'Grecia': 100.0,
 'Guatemala': 100.0,
 'Honduras': 100.0,
 'Haití': 100.0,
 'Indonesia': 100.0,
 'India': 100.0,
 'Irlanda': 100.0,
 'Irán, República Islámica del': 100.0,
 'Islandia': 100.0,
 'Israel': 100.0,
 'Italia': 100.0,
 'Jamaica': 100.0,
 'Japón': 100.0,
 'Kenya': 100.0,
 'Corea, República de': 100.0,
 'América Latina y el Caribe (excluido altos ingresos)': 100.0,
 'Sri Lanka': 100.0,
 'Luxemburgo': 100.0,
 'Marruecos': 100.0,
 'México': 100.0,
 'Malta': 100.0,
 'Malasia': 100.0,
 'América del Norte': 100.0,
 'Nigeria': 100.0,

### Order information

In [17]:
auxDataframe = pd.DataFrame(columns=CPIworld.columns)
print(auxDataframe)


for i, u in zip(nanCountryOrder, range(len(nanCountryOrder))):
    rowAdd = {x:np.nan for x in CPIworld.columns}
    rowSelected = CPIworld[CPIworld["Country"] == i]

    for w in auxDataframe.columns:
        rowAdd[w] = rowSelected[w].values[0]
    
    new_df = pd.DataFrame([rowAdd])
    auxDataframe = pd.concat([auxDataframe, new_df], ignore_index=True)
    
CPIworld = auxDataframe
CPIworld = CPIworld.round(2)
CPIworld

C:\Users\matia\AppData\Local\Temp\ipykernel_27640\1890376020.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  auxDataframe = pd.concat([auxDataframe, new_df], ignore_index=True)


Empty DataFrame
Columns: [Country, Code, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Index: []

[0 rows x 64 columns]


,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,AUS,3.73,2.29,-0.32,0.64,2.87,3.41,3.29,3.48,...,1.76,2.45,2.49,1.51,1.28,1.95,1.91,1.61,0.85,2.86
1,Austria,AUT,1.95,3.54,4.38,2.71,3.87,4.93,2.05,3.97,...,2.49,2.00,1.61,0.90,0.89,2.08,2.00,1.53,1.38,2.77
2,Bélgica,BEL,0.30,0.99,1.40,2.15,4.17,4.07,4.17,2.73,...,2.84,1.11,0.34,0.56,1.97,2.13,2.05,1.44,0.74,2.44
3,Burkina Faso,BFA,7.79,18.56,1.68,5.57,1.85,-0.73,2.36,-4.33,...,3.82,0.53,-0.26,0.72,0.44,1.48,1.96,-3.23,1.88,3.65
4,Bolivia,BOL,11.53,7.56,5.88,-0.71,10.18,2.86,6.95,11.20,...,4.52,5.74,5.77,4.06,3.62,2.82,2.27,1.84,0.94,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Venezuela,VEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.07,40.64,62.17,121.74,254.95,NaN,NaN,NaN,NaN,NaN
235,Islas Caimán,CYM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.19,2.16,1.27,-2.35,-0.63,NaN,NaN,NaN,NaN,NaN
236,Nueva Caledonia,NCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.75,1.28,0.18,0.57,0.58,NaN,NaN,NaN,NaN,NaN
237,Nauru,NRU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### <font color="orange">Countrys that need more data</font>

In [18]:
CPIworld_extra = pd.DataFrame()

# <font color="green">Save changes</font>

In [19]:
CPIworld.to_csv("CPIworld.csv")

In [20]:
CPIworld[CPIworld["Country"] == "Mundo"]

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
144,Mundo,WLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.73,2.62,2.35,1.43,1.55,2.19,2.44,2.19,1.92,3.5


In [21]:
CPIworld

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,AUS,3.73,2.29,-0.32,0.64,2.87,3.41,3.29,3.48,...,1.76,2.45,2.49,1.51,1.28,1.95,1.91,1.61,0.85,2.86
1,Austria,AUT,1.95,3.54,4.38,2.71,3.87,4.93,2.05,3.97,...,2.49,2.00,1.61,0.90,0.89,2.08,2.00,1.53,1.38,2.77
2,Bélgica,BEL,0.30,0.99,1.40,2.15,4.17,4.07,4.17,2.73,...,2.84,1.11,0.34,0.56,1.97,2.13,2.05,1.44,0.74,2.44
3,Burkina Faso,BFA,7.79,18.56,1.68,5.57,1.85,-0.73,2.36,-4.33,...,3.82,0.53,-0.26,0.72,0.44,1.48,1.96,-3.23,1.88,3.65
4,Bolivia,BOL,11.53,7.56,5.88,-0.71,10.18,2.86,6.95,11.20,...,4.52,5.74,5.77,4.06,3.62,2.82,2.27,1.84,0.94,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Venezuela,VEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.07,40.64,62.17,121.74,254.95,NaN,NaN,NaN,NaN,NaN
235,Islas Caimán,CYM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.19,2.16,1.27,-2.35,-0.63,NaN,NaN,NaN,NaN,NaN
236,Nueva Caledonia,NCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.75,1.28,0.18,0.57,0.58,NaN,NaN,NaN,NaN,NaN
237,Nauru,NRU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
